In [1]:
# change working dir
import os
os.chdir('../')

import torch
from src.datasets.scannet import ScanNetDataset
from src.threedsam.backbone import build_backbone
from src.threedsam.threedsam_modules.structure_extract import StructureExtractor
from src.threedsam.utils.supervision import spvs_coarse
from src.threedsam.utils.index_padding import anchor_index_padding
from src.config.default import get_cfg_defaults
from src.utils.misc import lower_config

root_dir = '/media/morgen/T7/dataset/scannet_train'
npz_path = '/media/morgen/T7/dataset/scannet_indices/scene_data/train/scene0066_00.npz'
intrinsic_path = '/media/morgen/T7/dataset/scannet_indices/intrinsics.npz'
dpt_weight_path = '/home/morgen/projects/3DSAM/weights/dpt/dpt_hybrid_nyu-2ce69ec7.pt'
dataset = ScanNetDataset(root_dir, npz_path, intrinsic_path, dpt_weight_path)

/home/morgen/.conda/envs/3dsam/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/morgen/.conda/envs/3dsam/lib/python3.8/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [2]:
sample = dataset[10]
sample.keys()

config = get_cfg_defaults()
config_SE = lower_config(config['THREEDSAM']['EXTRACTOR'])

In [3]:
extra_data = dict(
    hw0_i = (480, 640),
    hw1_i = (480, 640),
    hw0_c_16 = (30, 40), 
    hw1_c_16 = (30, 40),
    hw0_c = (60, 80),
    hw1_c = (60, 80),
)

In [4]:
sample['image0'] = sample['image0'].unsqueeze(0).cuda()
sample['image1'] = sample['image1'].unsqueeze(0).cuda()
sample['depth0'] = sample['depth0'].unsqueeze(0).cuda()
sample['depth1'] = sample['depth1'].unsqueeze(0).cuda()
sample['pts_3d0'] = sample['pts_3d0'].unsqueeze(0).cuda()
sample['pts_3d1'] = sample['pts_3d1'].unsqueeze(0).cuda()
sample['T_0to1'] = sample['T_0to1'].unsqueeze(0).cuda()
sample['T_1to0'] = sample['T_1to0'].unsqueeze(0).cuda()
sample['K0'] = sample['K0'].unsqueeze(0).cuda()
sample['K1'] = sample['K1'].unsqueeze(0).cuda()

sample['pts_3d0'] = sample['pts_3d0'].view(1, 480*640, 3)
sample['pts_3d1'] = sample['pts_3d1'].view(1, 480*640, 3)

data = {**sample, **extra_data}

In [5]:
sample['pts_3d0'].shape

torch.Size([1, 480, 640, 3])

In [6]:
# 监督测试
spvs_coarse(data, config)

In [7]:
# 锚点真值填充
struct_extractor = StructureExtractor(config_SE)
struct_extractor = struct_extractor.cuda().eval()

In [8]:
N, L, C = 1, 4800, 256
data['match_mask'] = torch.eye(L).unsqueeze(dim=0).cuda()
feat0 = torch.randn((N, L, C), device='cuda:0')
feat1 = torch.randn((N, L, C), device='cuda:0')

aug_feat0, aug_feat1 = struct_extractor(feat0, feat1, data)

/opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [8,0,0], thread: [64,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [8,0,0], thread: [65,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [8,0,0], thread: [66,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [8,0,0], thread: [67,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
